In [1]:
# initial imports
import numpy as np
import pymc as pc
import pandas as pd

# to sanitize the data
import nltk
from nltk.tag import pos_tag
from nltk import word_tokenize

nltk.download("maxent_treebank_pos_tagger")
nltk.download("maxent_ne_chunker")
nltk.download("punkt")
nltk.download(["tagsets", "universal_tagset"])
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_treebank_pos_tagger to
[nltk_data]     C:\Users\Irikos\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_treebank_pos_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     C:\Users\Irikos\AppData\Roaming\nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Irikos\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package tagsets to
[nltk_data]     C:\Users\Irikos\AppData\Roaming\nltk_data...
[nltk_data]   Package tagsets is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Irikos\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Irikos\AppData\Roaming\nltk_data...
[

True

In [11]:
# defining all the functions here

# read the csv, cast it to lowercase, keep only the nouns and make a list of lists out of it
def read_and_sanitize(document):
    df = pd.read_csv(document)
    df.drop('url', inplace=True, axis=1)
    df['text'] = df['text'].str.lower()
    df = df['text'].values
    
    test = []
    for document in df:
        documentAsList = nltk.word_tokenize(document)
        test.append(documentAsList)
    return test
#         documentPosTag = nltk.pos_tag(documentAsList)
#         newDf = []
#         for index, document in enumerate(documentPosTag):
#             for pos, tag in document.items():
#                 if tag == 'CD' or tag == 'NN' or tag == 'NNP':
#                     newDf[index].append(tag)
    

# read the document (list of lists) and assing an id for each word
def create_words_dictionary(documents_collection):
    words_index_dict =  {}
    index = 0
    for document in documents_collection:
        for word in document:
            if (word not in words_index_dict):
                words_index_dict[word] = index
                index += 1
    return words_index_dict

# replace all the words in the initial document with the ids
def replace_words_with_index(words_index_dict, documents_collection):
    my_collection = []
    for document in documents_collection:
        document_list = []
        for word in document:
            document_list.append(words_index_dict[word])
        my_collection.append(document_list)
    return my_collection

In [19]:
df = pd.read_csv('good.csv')
df.drop('url', inplace=True, axis=1)
df['text'] = df['text'].str.lower()

df = df['text'].values
test = []
for document in df:
    documentAsList = nltk.word_tokenize(document)
    test.append(documentAsList)
print (test)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
my_list2 = [2, 3, 4, 5]

In [12]:
# documents source: https://www.kaggle.com/urbanbricks/wikipedia-promotional-articles#good.csv
clean_data = read_and_sanitize('good.csv')

# noun = []
# for  index, row in df.iterrows():
#     noun.append([word for word,pos in pos_tag(row) if pos == 'NN'])
# print(noun.head())

# gonna play with
my_list = clean_data[0:2]

print(my_list)


KeyboardInterrupt: 

In [75]:
words_index_dictionary = create_words_dictionary(my_list)
# print(my_dictionary)
indexed_text = replace_words_with_index(words_index_dictionary, my_list)
#print(my_clean_doc)

print(3)

3


In [101]:
# number of topics
topics_no =  10
# number of words
words_size = len(words_index_dictionary)

#number of documents
documents_no = len(indexed_text)

alpha = np.ones(topics_no)
beta = np.ones(V)

# length of each list 
documents_length = [len(document) for document in indexed_text]

print (documents_length)

[3466, 3230]


In [96]:
# Calculate theta, topic distribution in each document  
theta = []
for index in range(documents_no):
    dirichlet = pc.Dirichlet("ptheta_%s"%index, theta=alpha)
    theta.append(pc.CompletedDirichlet("theta_%s" % index, dirichlet))

# print(theta)
print(2)

2


In [103]:
# Calculate phi, word distribution in each topic
phi = []
for index in range(topics_no):
    dirichlet = pc.Dirichlet("pphi_%s" % index, theta=beta)
    phi.append(pc.CompletedDirichlet("phi_%s" % index, dirichlet))

# print(phi)
print(2)

2


In [120]:
# topics
z = []
for index in range(documents_no):
    z.append(pc.Categorical("z_%i" % index, 
                            p = theta[index], 
                            size = documents_length[index], 
                            value = np.random.randint(documents_no, size=documents_length[index])))

print(z)

[<pymc.distributions.Categorical 'z_0' at 0x000001CB9EA53828>, <pymc.distributions.Categorical 'z_1' at 0x000001CB9EA53E10>]


In [ ]:
# the word generate from phi, using a topic z
w = []

for index in range(documents_no):
    for jindex in range(documents_length[index]):
        w.append(pc.Categorical("w_%i_%i" % (index, jindex),  
                          p = pc.Lambda("phi_z_%i_%i" % (index,jindex),  
                          lambda z = z[index][jindex], phi = phi : phi[z]),
                          value = indexed_text[index][jindex],  
                          observed = True))
# print (w)
print (2)


2


In [124]:
model = pc.Model([theta, phi, z, w])
mcmc = pc.MCMC(model)  
mcmc.sample(iter=50, burn=10)  

for d in range(documents_no):  
    print(mcmc.trace('z_%i'%d)[39])

TypeError: hasattr(): attribute name must be string

[2, 3, 4]
